***Vorlesung 'Syntax natürlicher Sprachen'***

--- 
# Zusatz: Relativsätze *center-embedding'-Rekursion


In [1]:
import nltk


## Rekursion in Phrasenstrukturgrammatik: *center-embedding* von Relativsätzen

In [2]:
grammar = nltk.CFG.fromstring("""
## Syntaktische Regeln:
    S -> NP VP
    NP -> Det N
    NP -> Det N Relpron NP VP
    VP -> V
##Lexikalische Regeln:
    Det -> 'die'
    Relpron -> 'welche'
    N -> 'Katze'
    V -> 'erschrickt' 
""")

parser = nltk.ChartParser(grammar,trace=0)

In [3]:
#Generierung:
from nltk.parse.generate import generate
for sentence in generate(grammar, depth=6):
    print(' '.join(sentence))

die Katze erschrickt
die Katze welche die Katze erschrickt erschrickt
die Katze welche die Katze welche die Katze erschrickt erschrickt erschrickt


In [4]:
#mit erweiterten lexikalischen Regeln:
grammar = nltk.CFG.fromstring("""
## Syntaktische Regeln:
    S -> NP VP
    NP -> Det N
    NP -> Det N Relpron NP VP
    VP -> V
##Lexikalische Regeln:
    Det -> 'die'
    Relpron -> 'welche'
    N -> 'Katze' | 'Maus'
    V -> 'sieht' | 'beobachtet' | 'erschrickt' 
""")

parser = nltk.ChartParser(grammar,trace=0)

In [5]:
sent = 'die Maus erschrickt'.split()
for tree in parser.parse(sent):
    print(tree)
    tree.pretty_print(unicodelines=True)

(S (NP (Det die) (N Maus)) (VP (V erschrickt)))
         S             
     ┌───┴───────┐      
     NP          VP    
 ┌───┴───┐       │      
Det      N       V     
 │       │       │      
die     Maus erschrickt



In [6]:
sent = 'die Maus welche die Katze sieht erschrickt'.split()
for tree in parser.parse(sent):
    #print(tree)
    tree.pretty_print(unicodelines=True)

                  S                            
            ┌─────┴──────────────────────┐      
            NP                           │     
 ┌───┬──────┼─────────┬──────────┐       │      
 │   │      │         NP         VP      VP    
 │   │      │     ┌───┴────┐     │       │      
Det  N   Relpron Det       N     V       V     
 │   │      │     │        │     │       │      
die Maus  welche die     Katze sieht erschrickt



In [7]:
##Center-Embedding:
sent = 'die Maus welche die Katze welche die Maus beobachtet sieht erschrickt'.split()
for tree in parser.parse(sent):
    #print(tree)
    tree.pretty_print(unicodelines=True)

                              S                                            
                       ┌──────┴──────────────────────────────────────┐      
                       NP                                            │     
 ┌───┬──────┬──────────┴──────┬──────────────────────────────┐       │      
 │   │      │                 NP                             │       │     
 │   │      │     ┌────┬──────┼─────────┬───────────┐        │       │      
 │   │      │     │    │      │         NP          VP       VP      VP    
 │   │      │     │    │      │     ┌───┴───┐       │        │       │      
Det  N   Relpron Det   N   Relpron Det      N       V        V       V     
 │   │      │     │    │      │     │       │       │        │       │      
die Maus  welche die Katze  welche die     Maus beobachtet sieht erschrickt



## Dependenzanalyse:

In [8]:
grammar = nltk.DependencyGrammar.fromstring("""
    'erschrickt' -> 'Maus'
    'Katze' -> 'die' | 'beobachtet'
    'Maus' -> 'die' | 'sieht'
    'sieht' -> 'welche' | 'Katze'
    'beobachtet' -> 'welche' | 'Maus'
    """)

parser = nltk.ProjectiveDependencyParser(grammar)

In [9]:
sent = 'die Maus welche die Katze sieht erschrickt'.split()

for tree in parser.parse(sent):
    tree.pretty_print(unicodelines=True)

    erschrickt            
        │                  
       Maus               
 ┌──────┴────────┐         
 │             sieht      
 │      ┌────────┴─────┐   
 │      │            Katze
 │      │              │   
die   welche          die 



In [10]:
sent = 'die Maus welche die Katze welche die Maus beobachtet sieht erschrickt'.split()

for tree in parser.parse(sent):
    tree.pretty_print(unicodelines=True)

           erschrickt                             
               │                                   
              Maus                                
 ┌─────────────┴────────┐                          
 │                    sieht                       
 │    ┌─────────────────┴─────┐                    
 │    │                     Katze                 
 │    │        ┌──────────────┴────────┐           
 │    │        │                   beobachtet     
 │    │        │              ┌────────┴───────┐   
 │    │        │              │               Maus
 │    │        │              │                │   
die welche    die           welche            die 

